### Best-First Search in the Blocksworld###

Goals of this lab
1. Some Python programming
1. A search example and code that's like the Pacman search assignment but not exactly
1. Show the various components of a search problem broken up into classes/modules (the Pacman code does not do that so well)
1. Blocksworld-like problems have been an important part of AI research since the beginning, and for good reason

------------------------------------------------

#### The Blocksworld Scenario ####

* There are two surfaces, the *table* and the *stack*
  * Blocks on the *table* are not stacked on each other
  * There is exactly one stack of 0 or more blocks on the *stack*
* There are five blocks, named <b>A</b> through <b>E</b>.
* There is a single gripper, which 
  * Can hold up to one block
  * Is located either at the *table* or at the *stack*
  
The operators available are

* **pickup(block)**
  * Precondition
    * The gripper must be empty
    * The argument must be the name of a block
    * The block must be at the same location as the gripper
    * The block must not have another block on top of it
  * Postcondition
    * The gripper is holding the block
    * The block is no longer at its previous position

* **putdown(block)**
  * Precondition
    * The gripper must be holding a block
    * The argument must be the name of a block
  * Postcondition
    * The gripper is empty
    * The block is at the location of the gripper
      * If the location is the *stack*, the block is at the top of the stack

* **move(surface)**
  * Precondition
    * The argument must be the name of a surface
  * Postcondition
    * The gripper is located at that surface


---------------------------------------------------

****Initial and Goal States****

The system should be flexible on handling various initial states and goals, but for this class we will consider one initial state and two goal states, one easy and one hard.  (Why would the hard one be hard?)

**Initial** ![Initial state](blocksworldInitial.GIF)
**Easy Goal** ![Easy goal](blocksworldGoalEasy.GIF)
**Difficult Goal** ![Difficult goal](blocksworldGoalHard.GIF) 

****The World State Description****

We need some data structure that holds the state of the world at some point in time.  It is important that this representation be compact because there will be many copies in the search queue.  For now we will be moderately thrifty with space.  Remember we need to capture
* What the gripper is holding
* Where the gripper is
* The blocks on the table (in any order)
* The blocks on the stack (in stacked order)

The world state class also has the operator definitions -- so it describes the "physics" of the blocks world.

In [ ]:
import copy

class BlocksWorldState:

    def __str__(self):
        return "{" + str(self._table) + "|" + str(self._stack) + "|" + str(self._gripperHolding) + "|" + self._gripperLocation + "}"
    
    def __eq__(self, other):
        if isinstance(other, BlocksWorldState):
            return self._table == other._table and self._stack == other._stack \
                and self._gripperHolding == other._gripperHolding and self._gripperLocation == other._gripperLocation
        else:
            return False

    def __hash__(self):
        return hash(str(self._table) + str(len(self._table)) + str(self._gripperHolding) + str(self._stack) + \
                                           str(len(self._stack)) + str(self._gripperLocation))
    
    def successors(self):
        candidates = (self.pickup(), self.putdown(), self.move())
        candidates = [val for sublist in candidates for val in sublist]
        return candidates
    
    def pickup(self):
     raise "Not implemented"
            
    def putdown(self):
     raise "Not implemented"
            
    def move(self):
        s = copy.deepcopy(self)
        s._gripperLocation = "S" if s._gripperLocation == "T" else "T"
        return [(s, "move" + s._gripperLocation)]   


****Problem Definitions and Evaluators****

* A problem definition is an initial state and a goal in the world, so it knows about world states, but not vice versa.  The problem definition doesn't have any knowledge of how it will be solved
* A problem evaluator has the cost and estimation functions that will be used by the search algorithm

In [ ]:
class EasyBlocksWorldProblem:
    @staticmethod
    def initial():
        s = BlocksWorldState()
        s._table = ["A", "C", "D", "E"]
        s._stack = ["B"]
        s._gripperHolding = None
        s._gripperLocation = "T"
        return s
    @staticmethod
    def isGoal(state):
        return state._stack == ["D", "C"]

class EasyBlocksWorldProblemEvaluator:
    @staticmethod
    def estimateToGoal(state):
        stack = state._stack
        if stack == ["D", "C"]:
            return 0
        if len(stack) == 0:
            return 2
        if stack[0] == "D":
            if len(stack) == 1:
                return 1
            else:
                return 4
        else:
            return 5
    @staticmethod
    def costSoFar(actions):
        return len(actions)
    @staticmethod
    def value(state, actions):
        return EasyBlocksWorldProblemEvaluator.costSoFar(actions) + EasyBlocksWorldProblemEvaluator.estimateToGoal(state)

##   I had to mess around with the evaluator to make it a hard problem!

class HardBlocksWorldProblem:
    @staticmethod
    def initial():
        s = BlocksWorldState()
        s._table = ["A", "C", "D", "E"]
        s._stack = ["B"]
        s._gripperHolding = None
        s._gripperLocation = "T"
        return s
    @staticmethod
    def isGoal(state):
        return state._stack == ["A", "B"]
 
class HardBlocksWorldProblemEvaluator:
    @staticmethod
    def estimateToGoal(state):
        stack = state._stack
        if stack == ["A", "B"]:
            return 0
        if len(stack) == 0:
            return 2
        if "A" in stack or "B" in stack:
            return 1
        else:
            return 3
    @staticmethod
    def costSoFar(actions):
        return len(actions)
    @staticmethod
    def value(state, actions):
        return HardBlocksWorldProblemEvaluator.costSoFar(actions) + HardBlocksWorldProblemEvaluator.estimateToGoal(state)

----------------------------------------------

In [ ]:
## This is what goes on the search fringe -- it is a pretty inert object!

class SearchState:
    def __str__(self):
        return "{S " + str(self._worldState) + "/" + str(self._actions) + "}"
    
    def __init__(self, worldState, actions):
        self._worldState = worldState
        self._actions = actions
    
    def worldState(self):
        return self._worldState
    
    def actions(self):
        return self._actions
    

In [ ]:
## Priority Queue copied from Berkeley AI Pacman Code 

import heapq

class PriorityQueue:
    """
      Implements a priority queue data structure. Each inserted item
      has a priority associated with it and the client is usually interested
      in quick retrieval of the lowest-priority item in the queue. This
      data structure allows O(1) access to the lowest-priority item.
    """
    def  __init__(self):
        self.heap = []
        self.count = 0

    def push(self, item, priority):
        entry = (priority, self.count, item)
        heapq.heappush(self.heap, entry)
        self.count += 1

    def pop(self):
        (_, _, item) = heapq.heappop(self.heap)
        return item

    def isEmpty(self):
        return len(self.heap) == 0

    def pp(self):
        print("["),
        for entry in self.heap:
            print(entry, " "),
        print("]")
    
    def update(self, item, priority):
        # If item already in priority queue with higher priority, update its priority and rebuild the heap.
        # If item already in priority queue with equal or lower priority, do nothing.
        # If item not in priority queue, do the same thing as self.push.
        for index, (p, c, i) in enumerate(self.heap):
            if i == item:
                if p <= priority:
                    break
                del self.heap[index]
                self.heap.append((priority, c, item))
                heapq.heapify(self.heap)
                break
        else:
            self.push(item, priority)


In [ ]:
## The search algorithm itself -- it takes a problem, which will give it an initial state and the goal test,
##   the world state itself which gives it the successor states, and an evaluator that evaluates the quality
##   of a node on the search frontier
import sys

def aStarSearch(problem, evaluator):
    raise "Not implemented"

        

In [ ]:
problem = EasyBlocksWorldProblem()
evaluator = EasyBlocksWorldProblemEvaluator()
print(aStarSearch(problem, evaluator))

In [ ]:
problem = HardBlocksWorldProblem()
evaluator = HardBlocksWorldProblemEvaluator()
print(aStarSearch(problem, evaluator))